<a href="https://colab.research.google.com/github/choi4490/AAA/blob/main/%EC%9E%91%EC%97%85%ED%98%952_%EA%B8%B0%EC%B6%9C%EB%AA%A8%EC%9D%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# # 7회 작업형 2 - 마트 판매액 예측하기

파일보고. 예측모델 판단하기
전처리> 인코딩여부 > 모델링 > 평가 > 제출
3. 어떤 항목이 판매액에 가장 큰 영향을 주는거야?

In [55]:
#데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/mart_train.csv") # (700,10)
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/mart_test.csv")   # (300,09)
train.head(10)    # dtypes: float64(2), object(8)
# train.corr(method = 'spearman') # 스피어만 상관 계수는 순위 기반 상관 관계를 계산하기 위해 숫자 데이터가 필요합니다.


,branch,city,customer_type,gender,product_line,total,payment_method,rating,time_of_day,day_name
0,A,Yangon,Member,Female,Health and beauty,823457.2500000000,Ewallet,9.1000000000,afternoon,Saturday
1,C,Naypyitaw,Normal,Female,Electronic accessories,120330.0000000000,Cash,9.6000000000,morning,Friday
2,A,Yangon,Normal,Male,Home and lifestyle,510788.2500000001,Credit card,7.4000000000,afternoon,Sunday
3,A,Yangon,Member,Male,Health and beauty,733572.0000000000,Ewallet,8.4000000000,evening,Sunday
4,A,Yangon,Normal,Male,Sports and travel,951567.7500000000,Ewallet,5.3000000000,morning,Friday
5,C,Naypyitaw,Normal,Male,Electronic accessories,941424.7500000000,Ewallet,4.1000000000,evening,Monday
6,A,Yangon,Member,Female,Electronic accessories,650538.0000000000,Ewallet,5.8000000000,afternoon,Monday
7,C,Naypyitaw,Normal,Female,Home and lifestyle,1158570.0000000000,Ewallet,8.0000000000,morning,Sunday
8,A,Yangon,Member,Female,Health and beauty,114219.0000000000,Credit card,7.2000000000,evening,Thursday
9,B,Mandalay,Member,Female,Food and beverages,259119.0000000000,Credit card,5.9000000000,afternoon,Wednesday


In [40]:
train.describe(include='object')    # 'O' ='object'

,branch,city,customer_type,gender,product_line,payment_method,time_of_day,day_name
count,700,700,700,700,700,700,700,700
unique,3,3,2,2,6,3,3,7
top,A,Yangon,Normal,Male,Sports and travel,Cash,evening,Saturday
freq,236,236,354,356,127,246,309,114


In [41]:
train.isnull().sum().sum()
train['total'].value_counts()

,count
total,
283641.7500000000,2
263875.5000000000,2
415422.0000000000,2
326450.2500000000,2
130851.0000000000,2
...,...
293391.0000000000,1
137103.7500000000,1
348232.5000000000,1


In [42]:
pd.set_option('display.float_format', '{:.10f}'.format)
train['total'].describe()

,total
count,700.0000000000
mean,485078.0175000000
std,364390.7265411940
min,19041.7500000000
25%,200119.5000000000
50%,381874.5000000000
75%,706127.6250000000
max,1563975.0000000002


In [43]:
target = train.pop('total')    # Train 자체학습해야하므로 트레인에서는 빼주고. target 테이블을 새로생성하자.
train.shape, target.shape

((700, 9), (700,))

In [58]:
print(train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)  #원핫인코딩이네.. 그냥 숫자로 다 변경해버린다.
print(train.shape, test.shape)
train.corr(method = 'spearman')  # 절대값처리 쉽네. .abs()

(700, 31) (300, 30)
(700, 31) (300, 30)


,total,rating,branch_A,branch_B,branch_C,city_Mandalay,city_Naypyitaw,city_Yangon,customer_type_Member,customer_type_Normal,...,time_of_day_afternoon,time_of_day_evening,time_of_day_morning,day_name_Friday,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
total,1.0000000000,-0.0181270339,0.0082475237,-0.0225953107,0.0143126162,-0.0225953107,0.0143126162,0.0082475237,0.0457150363,-0.0457150363,...,0.0009123353,0.0452530278,-0.0592746607,-0.0150674670,-0.0127502925,0.0136419571,0.0293785472,-0.0168437026,0.0065317697,-0.0060660539
rating,-0.0181270339,1.0000000000,0.0125639089,-0.0650551597,0.0524376732,-0.0650551597,0.0524376732,0.0125639089,0.0014637306,-0.0014637306,...,0.0168990733,0.0000284789,-0.0212093929,0.0508507357,0.0148097820,-0.0506602134,0.0434445054,-0.0231412435,0.0446569415,-0.0785489834
branch_A,0.0082475237,0.0125639089,1.0000000000,-0.5021322058,-0.5021322058,-0.5021322058,-0.5021322058,1.0000000000,-0.0160259322,0.0160259322,...,-0.0041716923,-0.0436784301,0.0613355886,0.0275064759,0.0519238406,-0.0281072423,0.0324586056,0.0149608692,-0.0562455163,-0.0348431280
branch_B,-0.0225953107,-0.0650551597,-0.5021322058,1.0000000000,-0.4957264957,1.0000000000,-0.4957264957,-0.5021322058,0.0383973288,-0.0383973288,...,0.0302931784,0.0158206257,-0.0582772734,0.0242857005,-0.0536696732,0.0346615676,-0.0699456484,-0.0347121651,0.0407976468,0.0477249463
branch_C,0.0143126162,0.0524376732,-0.5021322058,-0.4957264957,1.0000000000,-0.4957264957,1.0000000000,-0.5021322058,-0.0223030555,0.0223030555,...,-0.0261036963,0.0280440672,-0.0033198755,-0.0519094753,0.0015244080,-0.0064344645,0.0373486260,0.0196874966,0.0156877235,-0.0127332328
city_Mandalay,-0.0225953107,-0.0650551597,-0.5021322058,1.0000000000,-0.4957264957,1.0000000000,-0.4957264957,-0.5021322058,0.0383973288,-0.0383973288,...,0.0302931784,0.0158206257,-0.0582772734,0.0242857005,-0.0536696732,0.0346615676,-0.0699456484,-0.0347121651,0.0407976468,0.0477249463
city_Naypyitaw,0.0143126162,0.0524376732,-0.5021322058,-0.4957264957,1.0000000000,-0.4957264957,1.0000000000,-0.5021322058,-0.0223030555,0.0223030555,...,-0.0261036963,0.0280440672,-0.0033198755,-0.0519094753,0.0015244080,-0.0064344645,0.0373486260,0.0196874966,0.0156877235,-0.0127332328
city_Yangon,0.0082475237,0.0125639089,1.0000000000,-0.5021322058,-0.5021322058,-0.5021322058,-0.5021322058,1.0000000000,-0.0160259322,0.0160259322,...,-0.0041716923,-0.0436784301,0.0613355886,0.0275064759,0.0519238406,-0.0281072423,0.0324586056,0.0149608692,-0.0562455163,-0.0348431280
customer_type_Member,0.0457150363,0.0014637306,-0.0160259322,0.0383973288,-0.0223030555,0.0383973288,-0.0223030555,-0.0160259322,1.0000000000,-1.0000000000,...,0.0354158586,-0.0099795599,-0.0315528006,-0.0350268027,-0.0086857549,0.0127795900,-0.0081536900,-0.0212189259,0.0403704523,0.0168908159
customer_type_Normal,-0.0457150363,-0.0014637306,0.0160259322,-0.0383973288,0.0223030555,-0.0383973288,0.0223030555,0.0160259322,-1.0000000000,1.0000000000,...,-0.0354158586,0.0099795599,0.0315528006,0.0350268027,0.0086857549,-0.0127795900,0.0081536900,0.0212189259,-0.0403704523,-0.0168908159


In [45]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((560, 30), (140, 30), (560,), (140,))

In [46]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)**0.5

In [52]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr, y_tr)    # Train테이블로 학습한 결과 모델
pred = rf.predict(X_val)  # 이 rf모델------------------------
pred[:5]
# rmse(y_val, pred)

array([578015.7075, 441304.605 , 595106.82  , 479212.965 , 504032.76  ])

최종예측하고. 결과파일 생성

In [60]:
final_pred = rf.predict(test)  # test 역시 원핫인코딩된 상태임.이걸로 그냥 하는건가?
final_pred[:5]   # 결과값만 있는 상태
제출물=pd.DataFrame({'total':final_pred })  # total 칼럼 아래에 fial_pred값을 넣어준다
제출물.to_csv('submission.csv', index=False)  #칼럼제목 빼려면 index=False
# 제출완료. 파일명은 submission.csv

# 6회-작업형2 문제 : 난방 부하 단계를 예측해주세요!(다중분류)easy
- 예측할 값(y): Heat_Load (Very Low, Low, Medium, High, Very High)
- 평가: f1-macro
- data: train.csv, test.csv
- 제출 형식: result.csv파일을 아래와 같은 형식으로 제출